<a href="https://colab.research.google.com/github/nirezuluet/dp/blob/main/for_Ayoub.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

In [ ]:
"To execute the code run each cell and follow the instructions"

In [108]:
"""
these are the instructions to submit your xlsx file
first: to the lef you have something called table of contents, clinck on the folder icon
second: there is a bar with 4 elements (the last one is an eye). Press the first one, which is a file icon
with an arrow
3rd: submit your file (and wait until it's charged)
4th: move the cursor to the file you just submitted, you will see 3 dots,
press the dots, you will have a menu of options, and select "Copy path"
5th: now copy the path between the symbols "" in this line 'excel_file = ""'
"""
excel_file = "/content/data.xlsx"
file_ = pd.read_excel(excel_file, sheet_name='Sheet1')
table = file_.drop(0)
table

,Well,Date Test,BFPD,BSW %,BOPD,BWPD
1,KLM-036ST2,2023-07-18,1452.0,65.6,500.0,952.0
2,KLM-039,2023-07-20,1322.0,80.7,256.0,1067.0
3,KLM-040,2023-07-23,1127.0,81.9,204.0,922.0
4,KLM-404,2023-07-23,395.0,80.4,78.0,318.0
5,KLM-401L,2023-07-25,152.0,52.0,73.0,79.0
...,...,...,...,...,...,...
82,BVM-011,2023-09-03,363.0,40.3,217.0,147.0
83,BVM-051,2023-09-03,1186.0,92.4,90.0,1096.0
84,DKM-008,2023-09-03,3528.0,77.2,805.0,2723.0
85,KLM-012ST,2023-09-03,1386.0,98.0,28.0,1359.0


In [109]:
# dynamic programming for the min-knapsack problem

"""
This problem can be modeled as a minimisation version of the knapsack problem
In this context:
- I have a set of items with a given volume (water production)
- These items have also associated a cost (oil production)
- Goal: minimise the total cost of selecting items while respecting a volume bigger than a given threshold

Return:
- The minimum total cost
- The indexes used to achive the minimal cost (obtained by backtracking the solution)
"""

def knapsack(n, costs, weights, target):
  MAX = float('inf')
  dp = [[0 for _ in range(n + 1)] for _ in range(target + 1)]
  #print(len(dp))
  for t in range(target + 1):
      dp[t][0] = MAX
  for i in range(n + 1):
      dp[0][i] = MAX

  for t in range(1, target + 1):
      for i in range(1, n + 1):
          if t > weights[i - 1]:  # i - 1 because of the offset
              dp[t][i] = min(dp[t][i - 1], dp[t - weights[i - 1]][i - 1] + costs[i - 1])
          else:
              dp[t][i] = min(dp[t][i - 1], costs[i - 1])


  # backtracking the solution to find the indexes
  subset_index = []
  j = n + 1
  t = target
  while t >=1:
    min_index = np.argmin(dp[t][:j]) #find the minimum from left to right just reaching given index j
    subset_index.append(min_index - 1)
    j = min_index
    t = t - weights[min_index - 1] #it could only came from above

  return min(dp[target]), subset_index


In [110]:
# getting the name and BFPD values
name = table["Well"].tolist()
production_oil = table["BOPD"].tolist()
production_water = table["BWPD"].tolist()

In [111]:
indexes = np.flip(np.argsort(production_water))
water_prod2 = [int(production_water[i]) for i in indexes]
oil_prod2 = [int(production_oil[i]) for i in indexes]
name_well2 = [name[i] for i in indexes]

threshold = 50000 #here to define the minum volume capacity (you can change the number here)
result = knapsack(len(oil_prod2), oil_prod2, water_prod2, threshold)

In [112]:
indexes = result[1]
oil_prodd = np.sum([oil_prod2[i] for i in indexes])
water_prodd = np.sum([water_prod2[i] for i in indexes])
names = [name_well2[i] for i in indexes]

In [113]:
print(f"Minimum oil production {oil_prodd}")
print(f"Total water production of the selected wells {water_prodd}")
print(f"Selected wells {names}")

Minimum oil production 2159
Total water production of the selected wells 50087
Selected wells ['BVM-031', 'BVM-032', 'BVM-037', 'BVM-047', 'ITM-109L', 'ASM-031', 'KLM-004ST', 'KLM-012ST', 'ITM-101L', 'KLM-037', 'ITM-103L', 'NNM-018', 'NNM-020', 'ACM-014', 'ITM-110L', 'LSM-104', 'ITM-102S', 'LSM-102', 'DIM-005ST', 'DKM-010', 'ASM-035']


In [114]:
# seing the results in a dataframe and asserting that we did not make any mistake
t = table[table['Well'].isin(names)]
t

,Well,Date Test,BFPD,BSW %,BOPD,BWPD
7,DIM-005ST,2023-08-03,5675.0,94.9,288.0,5387.0
11,ITM-110L,2023-08-10,2544.0,93.4,168.0,2376.0
12,NNM-018,2023-08-10,2015.0,93.5,130.0,1885.0
15,ITM-102S,2023-08-12,3462.0,95.7,149.0,3314.0
16,ITM-103L,2023-08-12,1850.0,96.5,65.0,1785.0
20,DKM-010,2023-08-15,6467.0,97.2,181.0,6286.0
22,ITM-101L,2023-08-16,1432.0,96.0,57.0,1374.0
23,KLM-037,2023-08-16,1714.0,95.9,71.0,1643.0
31,LSM-104,2023-08-18,3264.0,98.5,48.0,3216.0
34,ITM-109L,2023-08-19,974.0,95.4,45.0,929.0


In [105]:
print(f"Total water production from the list {np.sum(t['BWPD'].tolist())}")
assert np.sum(t['BWPD'].tolist()) == water_prodd

Total water production from the list 50087.0


In [106]:
print(f"Total oil production from the list {np.sum(t['BOPD'].tolist())}")
assert np.sum(t['BOPD'].tolist()) == oil_prodd

Total oil production from the list 2159.0


In [107]:
# saving the table in a xlxs file
t.to_excel("output.xlsx", sheet_name='Sheet_name_1')

In [ ]:
# brute force implementation. Do not pay attention.
# IT TAKES YEARS TO RUN

In [ ]:
# for the recursion
#min_total_oil = 1000000
#name_well_list = []

In [ ]:
# brute force. It takes years to give a solution
# it is neccesary to optimize it
def best_combination(water_prod, oil_prod, name_well, threshold, selected_well,
                     total_water, total_oil):

    #stopping criteria
    if total_water >= threshold:
        global min_total_oil
        global name_well_list
        if total_oil < min_total_oil:
            min_total_oil = total_oil
            name_well_list = selected_well.copy()
            print(total_water, total_oil)
            print(name_well_list)

    else:

        if len(water_prod) > 1 and total_oil < min_total_oil:
            # take the well
            new_list = selected_well.copy()
            new_list.append(name_well[0])
            best_combination(water_prod[1:], oil_prod[1:],
                             name_well[1:], threshold,
                             new_list, total_water + water_prod[0], total_oil + oil_prod[0])

            # do not take the well
            best_combination(water_prod[1:], oil_prod[1:],name_well[1:], threshold,
                            selected_well, total_water, total_oil)

In [ ]:
# do not run. Brute force that takes years
"""
def get_best_combination(oil_prod, water_prod, name_well, threshold):
# objective: drop the well that produce the less and sum of water_prod >= threshold

    #sorting the list and getting also the indexes for sorting the
    # remaining lists (decreasing order)
    indexes = np.flip(np.argsort(water_prod))
    water_prod2 = [water_prod[i] for i in indexes]
    oil_prod2 = [oil_prod[i] for i in indexes]
    name_well2 = [name_well[i] for i in indexes]



    best_combination(water_prod2, oil_prod2, name_well2, threshold, [], 0, 0)
"""

In [ ]:
#get_best_combination(production_oil, production_water, name, threshold )

61975.0 9065.0
['ASM-035', 'DKM-010', 'OCM-001ST3', 'DKM-003', 'DIM-005ST', 'LSM-102', 'DIM-007', 'ITM-102S', 'DKM-004S', 'LSM-104', 'ACM-013', 'DKM-008', 'OAK-S1A', 'LSM-101', 'ITM-110L']
61941.0 9003.0
['ASM-035', 'DKM-010', 'OCM-001ST3', 'DKM-003', 'DIM-005ST', 'LSM-102', 'DIM-007', 'ITM-102S', 'DKM-004S', 'LSM-104', 'ACM-013', 'DKM-008', 'OAK-S1A', 'LSM-101', 'ACM-014']
61628.0 9000.0
['ASM-035', 'DKM-010', 'OCM-001ST3', 'DKM-003', 'DIM-005ST', 'LSM-102', 'DIM-007', 'ITM-102S', 'DKM-004S', 'LSM-104', 'ACM-013', 'DKM-008', 'OAK-S1A', 'LSM-101', 'NNM-020']
61384.0 8962.0
['ASM-035', 'DKM-010', 'OCM-001ST3', 'DKM-003', 'DIM-005ST', 'LSM-102', 'DIM-007', 'ITM-102S', 'DKM-004S', 'LSM-104', 'ACM-013', 'DKM-008', 'OAK-S1A', 'LSM-101', 'ITM-103L']
60973.0 8954.0
['ASM-035', 'DKM-010', 'OCM-001ST3', 'DKM-003', 'DIM-005ST', 'LSM-102', 'DIM-007', 'ITM-102S', 'DKM-004S', 'LSM-104', 'ACM-013', 'DKM-008', 'OAK-S1A', 'LSM-101', 'ITM-101L']
60958.0 8925.0
['ASM-035', 'DKM-010', 'OCM-001ST3', 'DKM-

KeyboardInterrupt: ignored